### Install import labraries

In [ ]:
# !conda install scikit-learn
# !conda update scikit-learn
# !conda install joblib 
# !conda update joblib 

In [10]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import sklearn
import sklearn.datasets
import pandas as pd
from keras.models import Sequential
from keras.utils import to_categorical
from keras.layers import Dense

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

Using TensorFlow backend.


### Reading datafile and cleaning

In [3]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


In [4]:
# Remove Space for `FALSE POSITIVE` category
mask = df["koi_disposition"] == "FALSE POSITIVE"
df.loc[mask, "koi_disposition"] = "False_Positive"

### Select features and predictor variables

In [6]:
# selecting x and y value for the machine learning model
X_value = df.drop("koi_disposition", axis=1)
y_value = df['koi_disposition']
print(X_value.shape)
print(y_value.shape)

(6991, 40)
(6991,)


### Splitting data into train and test sets & preprocessing

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_value, y_value, random_state=42, stratify=y_value)

In [8]:
from sklearn.preprocessing import MinMaxScaler

X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

C:\Users\Samita\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [11]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(y_train)

y_train_code = label_encoder.transform(y_train)
y_test_code = label_encoder.transform(y_test)

In [13]:
y_train_categorical = to_categorical(y_train_code)
y_test_categorical = to_categorical(y_test_code)
y_train_categorical.shape

(5243, 3)

### Train model

In [15]:
model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=40))
model.add(Dense(units=100, activation='relu', input_dim=40))
model.add(Dense(units=3, activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.


In [16]:
model.compile(optimizer='adam',
             loss='categorical_crossentropy',
             metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 100)               4100      
_________________________________________________________________
dense_2 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 303       
Total params: 14,503
Trainable params: 14,503
Non-trainable params: 0
_________________________________________________________________


In [17]:
model.fit(
X_train_scaled,
y_train_categorical,
epochs=60,
shuffle=True,
verbose=2
)

Instructions for updating:
Use tf.cast instead.
Epoch 1/60
 - 1s - loss: 0.5570 - accuracy: 0.7261
Epoch 2/60
 - 0s - loss: 0.3702 - accuracy: 0.8144
Epoch 3/60
 - 0s - loss: 0.3546 - accuracy: 0.8285
Epoch 4/60
 - 0s - loss: 0.3428 - accuracy: 0.8331
Epoch 5/60
 - 0s - loss: 0.3380 - accuracy: 0.8394
Epoch 6/60
 - 0s - loss: 0.3281 - accuracy: 0.8455
Epoch 7/60
 - 0s - loss: 0.3284 - accuracy: 0.8516
Epoch 8/60
 - 0s - loss: 0.3212 - accuracy: 0.8493
Epoch 9/60
 - 0s - loss: 0.3118 - accuracy: 0.8575
Epoch 10/60
 - 0s - loss: 0.3089 - accuracy: 0.8594
Epoch 11/60
 - 0s - loss: 0.3096 - accuracy: 0.8629
Epoch 12/60
 - 0s - loss: 0.3101 - accuracy: 0.8613
Epoch 13/60
 - 0s - loss: 0.3009 - accuracy: 0.8640
Epoch 14/60
 - 0s - loss: 0.3060 - accuracy: 0.8623
Epoch 15/60
 - 0s - loss: 0.2983 - accuracy: 0.8703
Epoch 16/60
 - 0s - loss: 0.2944 - accuracy: 0.8716
Epoch 17/60
 - 1s - loss: 0.3032 - accuracy: 0.8615
Epoch 18/60
 - 0s - loss: 0.2924 - accuracy: 0.8699
Epoch 19/60
 - 0s - loss:

### Model evaluation and Prediction

In [18]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"result_df Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

result_df Neural Network - Loss: 0.23981451954121447, Accuracy: 0.8907322883605957


In [19]:
encoded_predictions = model.predict_classes(X_test_scaled[:20])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)
Actual_label = list(y_test[:20])

In [20]:
result_df = pd.DataFrame({
    'Prediction': prediction_labels,
    'Actual': Actual_label
})
result_df

,Prediction,Actual
0,CANDIDATE,CANDIDATE
1,False_Positive,False_Positive
2,False_Positive,False_Positive
3,False_Positive,False_Positive
4,CANDIDATE,CANDIDATE
5,False_Positive,False_Positive
6,CANDIDATE,CANDIDATE
7,False_Positive,False_Positive
8,False_Positive,False_Positive
9,False_Positive,False_Positive


### Save Model

In [21]:
import joblib
filename = 'deeplearning_model.sav'
joblib.dump(model, filename)

['deeplearning_model.sav']